# High performance pandas

```py
mask = (x > .5) & (y < .5)
df[df[mask]]

```

under the hood


```py
tmp1 = (x > .5)
tmp2 = (y < .5)
mask = tmp1 & tmp2
```

use pd.eval(""), df.query() -> does elementwise operation with numexpr



In [29]:
import numpy as np
import pandas as pd


nrows, ncols = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.388542,0.819008,0.593150,1.187528,0.235936,-0.276185,-0.960213,-0.313465,0.308365,-0.743423,...,0.668494,0.218215,0.676136,0.272783,1.214675,-0.387191,-2.727717,0.752075,-0.345214,-1.140413
1,1.645423,1.460578,-0.984456,1.033536,0.375388,-0.085564,-0.781761,0.975832,0.017510,1.082314,...,1.933305,1.598368,-0.419811,0.675477,-1.190947,-0.384259,1.364035,0.355124,-1.031217,-0.866923
2,0.903164,-0.213844,-1.234716,-0.855772,0.183035,-0.442289,-0.052720,-0.916614,-0.391531,-0.825277,...,0.001275,-0.050957,-0.652704,1.324303,-0.396401,0.901041,-0.926224,0.747261,-0.404462,-1.147789
3,-0.398645,-0.847027,-0.993868,1.101827,0.476785,-2.232567,-0.413736,0.008494,-1.476608,-0.814060,...,1.081686,-0.281576,-0.604156,0.099178,0.973948,-0.890738,0.600707,-0.159173,-1.258744,1.583136
4,1.377391,-1.895910,-0.549297,-0.493103,1.661027,0.577382,-0.277150,-0.325008,0.641584,-1.130527,...,-1.809564,1.030088,0.172858,0.326625,1.140567,-1.299636,0.071577,0.310963,-1.245753,0.539961


In [30]:
df1.shape

(1000000, 100)

In [31]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [32]:
%timeit df1+df2+df3+df4
%timeit pd.eval("df1+df2+df3+df4")

558 ms ± 38.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
232 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [34]:
rolls = pd.DataFrame(np.random.randint(1, 6, (6, 3)), columns=["Die1", "Die2", "Die3"])

rolls

,Die1,Die2,Die3
0,3,5,3
1,2,2,5
2,4,4,2
3,4,4,2
4,2,4,4
5,5,2,4


In [35]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace=True)
rolls

,Die1,Die2,Die3,Sum
0,3,5,3,11
1,2,2,5,9
2,4,4,2,10
3,4,4,2,10
4,2,4,4,10
5,5,2,4,11


In [36]:
high = 8
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,3,5,3,11,True
1,2,2,5,9,True
2,4,4,2,10,True
3,4,4,2,10,True
4,2,4,4,10,True
5,5,2,4,11,True


In [37]:
# traditional way to filter from a dataframe
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum
0,3,5,3,11
1,2,2,5,9
2,4,4,2,10
3,4,4,2,10
4,2,4,4,10
5,5,2,4,11


## Query

- filter using query

In [39]:
rolls.query("Sum > @high & Die1 == 4")

,Die1,Die2,Die3,Sum
2,4,4,2,10
3,4,4,2,10


In [ ]:
df_os = pd.read_csv("../Data/athlete_events.csv")